## Step 0: Imports

In [1]:
from gerrychain import (GeographicPartition, 
                        Partition, 
                        Graph, 
                        MarkovChain,
                        proposals, 
                        updaters, 
                        constraints, 
                        accept, 
                        Election)
from gerrychain.constraints import (single_flip_contiguous, 
                                    no_more_discontiguous,
                                    Validator)
from gerrychain.updaters import Tally, cut_edges
from gerrychain.proposals import propose_random_flip, recom
from gerrychain.accept import always_accept

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from functools import partial

import networkx as nx
import maup
import geopandas

import gerrychain

In [2]:
pd.options.display.max_columns = None
%matplotlib inline

## Step 1: Setting up the initial districting plan

### Wrangling the shapefile

In [3]:
# Load the shapefile into a graph
graph = Graph.from_file("VA_precincts/VA_precincts.shp")

/usr/local/anaconda3/lib/python3.7/site-packages/gerrychain/graph/adjacency.py:68: UserWarning: Found overlaps among the given polygons. Indices of overlaps: {(892, 914), (893, 915), (887, 892), (887, 893)}
  overlaps
/usr/local/anaconda3/lib/python3.7/site-packages/gerrychain/graph/graph.py:261: UserWarning: NA values found in column G17DGOV!
  warnings.warn("NA values found in column {}!".format(column))


In [4]:
# Load the shapefile into a geo pandas dataframe
df = geopandas.read_file("VA_precincts/VA_precincts.shp")

In [5]:
df.head()

,precinct,locality,loc_prec,district,G18DHOR,G18DSEN,G18OHOR,G18OSEN,G18RHOR,G18RSEN,G17DGOV,G17DLTG,G17DATG,G17DHOD,G17RGOV,G17RLTG,G17RATG,G17RHOD,G17OHOD,G17OGOV,G16DPRS,G16RPRS,G16OPRS,G16DHOR,G16RHOR,G16OHOR,TOTPOP,NH_WHITE,NH_BLACK,NH_AMIN,NH_ASIAN,NH_NHPI,NH_OTHER,NH_2MORE,HISP,H_WHITE,H_BLACK,H_AMN,H_ASIAN,H_NHPI,H_OTHER,H_2MORE,VAP,HVAP,WVAP,BVAP,AMINVAP,ASIANVAP,NHPIVAP,OTHERVAP,2MOREVAP,CD_12,CD_16,HDIST_11,HDIST_REM,SENDIST,geometry
0,Glenkirk,Prince William County,Prince William County Glenkirk,Congressional District 1,980.0,1044.0,0.0,32.0,950.0,856.0,715.0,701.0,698.0,650.0,744.0,763.0,769.0,804.0,0.0,13.0,1083.0,1155.0,119.0,920.0,1320.0,78.0,3651.999992,2324.999994,304.000000,8.000000,591.999998,1.000000e+00,4.000000,102.000000,316.000000,209.000000,8.000000,6.311268e-08,1.000000e+00,3.000000e+00,74.000000,21.000000,2474.999995,215.000000,1606.999997,199.000000,7.000000,400.999998,1.000000e+00,4.0,41.000000,1.0,1.0,13,13,13,"POLYGON ((163199.056 308336.365, 163189.724 30..."
1,Buckland Mills,Prince William County,Prince William County Buckland Mills,Congressional District 1,938.0,978.0,0.0,31.0,766.0,701.0,656.0,654.0,647.0,601.0,596.0,603.0,609.0,644.0,0.0,11.0,1007.0,807.0,78.0,920.0,932.0,11.0,3818.000031,2133.000020,405.000005,8.000000,643.000003,1.190457e-07,10.000000,150.000001,469.000001,265.000001,12.000000,1.000000e+01,1.000000e+00,0.000000e+00,141.000000,40.000000,2675.000021,313.000000,1539.000014,288.000004,7.000000,449.000002,1.190457e-07,6.0,73.000000,1.0,1.0,13,13,13,"POLYGON ((162078.796 309095.892, 162068.091 30..."
2,Limestone,Prince William County,Prince William County Limestone,Congressional District 1,1471.0,1562.0,0.0,55.0,1232.0,1079.0,1072.0,701.0,1035.0,963.0,917.0,881.0,970.0,1027.0,0.0,25.0,1520.0,1425.0,175.0,1345.0,1621.0,111.0,4496.999984,2690.999995,504.999996,6.000000,558.999998,1.828088e-08,15.000000,159.999999,560.999997,314.999998,34.000000,2.200000e+01,6.000000e+00,2.084499e-07,126.999999,57.000000,2876.999990,345.999998,1743.999996,344.999997,5.000000,368.999999,1.828088e-08,9.0,59.000000,1.0,1.0,13,13,13,"POLYGON ((163554.036 308215.804, 163546.179 30..."
3,Mullen,Prince William County,Prince William County Mullen,Congressional District 1,1312.0,1333.0,0.0,40.0,344.0,293.0,826.0,832.0,818.0,788.0,225.0,242.0,254.0,279.0,0.0,21.0,1669.0,553.0,121.0,1506.0,626.0,106.0,8228.999859,2136.999965,1619.999966,14.999999,435.999993,1.000000e+00,29.000000,255.999997,3734.999938,1455.999975,64.999999,2.500000e+01,4.000000e+00,0.000000e+00,1978.999970,205.999995,5779.999904,2390.999962,1754.999973,1136.999977,13.999999,342.999994,1.000000e+00,15.0,123.999999,1.0,1.0,13,13,13,"POLYGON ((171765.688 311597.152, 171757.655 31..."
4,Sudley,Prince William County,Prince William County Sudley,Congressional District 1,727.0,737.0,0.0,44.0,520.0,467.0,477.0,491.0,471.0,460.0,439.0,448.0,468.0,474.0,0.0,25.0,883.0,700.0,114.0,810.0,756.0,75.0,3655.000023,1942.000004,290.000005,9.000000,176.000002,2.000000e+00,8.000001,102.000000,1126.000012,421.000007,15.000000,3.300000e+01,2.166012e-08,0.000000e+00,585.000003,72.000002,2707.000016,726.000007,1559.000004,221.000003,8.000000,134.000001,2.000000e+00,8.0,49.000000,1.0,1.0,13,13,29,"MULTIPOLYGON (((174200.152 312528.358, 174200...."


In [6]:
df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2439 entries, 0 to 2438
Data columns (total 57 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   precinct   2439 non-null   object  
 1   locality   2439 non-null   object  
 2   loc_prec   2439 non-null   object  
 3   district   2439 non-null   object  
 4   G18DHOR    2439 non-null   object  
 5   G18DSEN    2439 non-null   object  
 6   G18OHOR    2439 non-null   object  
 7   G18OSEN    2439 non-null   object  
 8   G18RHOR    2439 non-null   object  
 9   G18RSEN    2439 non-null   object  
 10  G17DGOV    2438 non-null   object  
 11  G17DLTG    2439 non-null   object  
 12  G17DATG    2439 non-null   object  
 13  G17DHOD    2439 non-null   object  
 14  G17RGOV    2439 non-null   object  
 15  G17RLTG    2439 non-null   object  
 16  G17RATG    2439 non-null   object  
 17  G17RHOD    2439 non-null   object  
 18  G17OHOD    2439 non-null   object  
 19  G17OGOV    2439 non

In [7]:
# Fix UserWarning: NA values found in column G17DGOV!
df.iloc[964, 10] = '0.0'

In [8]:
# Convert to int type
cols = ['G18DHOR', 'G18DSEN', 'G18OHOR', 'G18OSEN', 'G18RHOR', 'G18RSEN',
        'G17DGOV', 'G17DLTG', 'G17DATG', 'G17DHOD', 'G17RGOV', 'G17RLTG', 
        'G17RATG', 'G17RHOD', 'G17OHOD', 'G17OGOV', 'G16DPRS', 'G16RPRS', 
        'G16OPRS', 'G16DHOR', 'G16RHOR', 'G16OHOR']
df2 = df.copy()
for col in cols:
    df2[col] = df2[col].astype(float).astype(int)

In [9]:
df2.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2439 entries, 0 to 2438
Data columns (total 57 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   precinct   2439 non-null   object  
 1   locality   2439 non-null   object  
 2   loc_prec   2439 non-null   object  
 3   district   2439 non-null   object  
 4   G18DHOR    2439 non-null   int64   
 5   G18DSEN    2439 non-null   int64   
 6   G18OHOR    2439 non-null   int64   
 7   G18OSEN    2439 non-null   int64   
 8   G18RHOR    2439 non-null   int64   
 9   G18RSEN    2439 non-null   int64   
 10  G17DGOV    2439 non-null   int64   
 11  G17DLTG    2439 non-null   int64   
 12  G17DATG    2439 non-null   int64   
 13  G17DHOD    2439 non-null   int64   
 14  G17RGOV    2439 non-null   int64   
 15  G17RLTG    2439 non-null   int64   
 16  G17RATG    2439 non-null   int64   
 17  G17RHOD    2439 non-null   int64   
 18  G17OHOD    2439 non-null   int64   
 19  G17OGOV    2439 non

In [10]:
# Save to shapefile format
df2.to_file('VA_precincts_cleaned/va.shp')